# 📘 System Identification: Transfer Function Fitting> Fit dynamic system models to step response data⏱️ **20-30 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Engineering**---

## 🔬 Domain Background**Model:** $y(t) = K(1 - e^{-t/\tau})$ (First-order system)**Applications:** Control systems, process dynamics---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
import numpy as npimport jax.numpy as jnpfrom nlsq import curve_fitimport matplotlib.pyplot as pltdef first_order_step(t, K, tau):    return K * (1 - jnp.exp(-t/tau))

## Generate Step Response Data

In [ ]:
K_true, tau_true = 10.0, 2.0  # Gain, time constantt = np.linspace(0, 15, 50)y_true = first_order_step(t, K_true, tau_true)y = y_true + np.random.normal(0, 0.3, len(t))plt.plot(t, y, 'o', alpha=0.5, label='Measured response')plt.plot(t, y_true, 'r--', label='True system')plt.xlabel('Time (s)')plt.ylabel('Output')plt.legend()plt.tight_layout()
display(fig)
plt.close(fig)


## Identify System Parameters

In [ ]:
popt, pcov = curve_fit(first_order_step, t, y, p0=[9, 1.5])K_fit, tau_fit = poptperr = np.sqrt(np.diag(pcov))print(f'System Parameters:')print(f'  Gain (K):         {K_fit:.2f} ± {perr[0]:.2f}')print(f'  Time constant (τ): {tau_fit:.2f} ± {perr[1]:.2f} s')print(f'\nSettling time (5τ): {5*tau_fit:.1f} s')

## Key Insights1. **Gain K** = steady-state output2. **Time constant τ** = 63.2% rise time3. **5τ rule** for settling time---